In [112]:
# import os
import json
from collections import defaultdict
import gzip
import pandas as pd
from lxml import html,etree
import numpy as np
import ipywidgets as widgets

In [113]:
### load the meta data
data = []
with gzip.open('Dataset/meta_Software.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print("Total number of items in the dataset: ", len(data))

# first row of the list
print(data[0])

Total number of items in the dataset:  26790
{'category': [], 'tech1': '', 'description': [], 'fit': '', 'title': 'HOLT PHYSICS LESSON PRESENTATION CD-ROM QUICK CONCEPTS', 'also_buy': [], 'tech2': '', 'brand': 'HOLT. RINEHART AND WINSTON', 'feature': [], 'rank': '25,550 in Software (', 'also_view': [], 'main_cat': 'Software', 'similar_item': '', 'date': '</div>', 'price': '.a-box-inner{background-color:#fff}#alohaBuyBoxWidget .selected{background-color:#fffbf3;border-color:#e77600;box-shadow:0 0 3px rgba(228,121,17,.5)}#alohaBuyBoxWidget .contract-not-available{color:gray}#aloha-cart-popover .aloha-cart{height:auto;overflow:hidden}#aloha-cart-popover #aloha-cartInfo{float:left}#aloha-cart-popover #aloha-cart-details{float:right;margin-top:1em}#aloha-cart-popover .deviceContainer{width:160px;float:left;padding-right:10px;border-right:1px solid #ddd}#aloha-cart-popover li:last-child{border-right:0}#aloha-cart-popover .aloha-device-title{height:3em;overflow:hidden}#aloha-cart-popover .alo

In [114]:
# convert list into pandas dataframe

df = pd.DataFrame.from_dict(data)

print(len(df))
print(df)


pd.set_option('display.max_colwidth', 300)
pd.set_option('display.max_rows', 20 )

26790
                                                                                                                                                                                                                                                                                                          category  \
0                                                                                                                                                                                                                                                                                                               []   
1                                                                                                                                                                                                                                                                                                               []   
2                                                               

In [115]:
### load the for "Software" category:

data2 = []
with gzip.open('Dataset/meta_Software.json.gz') as f:
    for l in f:
        data2.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print("Total number of items in the dataset: ", len(data2))


Total number of items in the dataset:  26790


In [116]:
# convert list into pandas dataframe

df2 = pd.DataFrame.from_dict(data2)

# Features of the dataset
df2.columns

# Features are slightly different than 


Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2',
       'brand', 'feature', 'rank', 'also_view', 'main_cat', 'similar_item',
       'date', 'price', 'asin', 'imageURL', 'imageURLHighRes', 'details'],
      dtype='object')

In [117]:
# df2.head()

In [118]:
# Some products have multiple descriptions containing information of for instance the author.
# Perhaps only the first description string should be analyzed for our purpose? This seems to be primarily for the product.
# For now i have just merged all descriptions into a single string for analysis.

#df2.description = df2.description.apply(lambda x: "".join(x))

# Drop rows with no description (empty list)
df2 = df2[df2['description'].map(lambda d: len(d)) > 0]



# df2.description


In [119]:

df2.description = df2.description.apply(lambda x: [string for string in x if string != ""])


In [120]:
for i in range(3):

    print(df2.iloc[i].description[0])

<b>Latin rhythms that will get your kids singing in Spanish</b>
<b>Connect is the only integrated learning system that empowers students by continuously adapting to deliver precisely what they need, when they need it, how they need it, so that your class time is more engaging and effective.</b><br />
<i>Anatomy &amp; Physiology Revealed Cat</i> is the ultimate online interactive cat dissection experience. This state-of-the-art program uses cat photos combined with a layering technique that allows the student to peel away layers of the cat to reveal structures beneath the surface. <i>Anatomy &amp; Physiology Revealed Cat</i> also offers animations, histologic and radiologic imaging, audio pronunciations, and comprehensive quizzing. It can be used as part of any one or two semester undergraduate anatomy &amp; physiology or human anatomy course; <i>Anatomy &amp; Physiology Revealed Cat</i> is available stand-alone, or can be combined with any McGraw-Hill book.


In [121]:
df2.description = df2.description.apply(lambda x: ". ".join(x))




In [122]:
df2.loc[1:4]
pd.set_option('display.max_rows', 20)

In [123]:
df2.loc[1:4]

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
1,[],,"<b>Latin rhythms that will get your kids singing in Spanish</b>. <i>Sing, Watch, and Learn Spanish</i> helps your kids ages four through eight take a giant step in learning Spanish by combining two time-honored methods kids have always used to develop their language skills: Imitating other kids ...",,"Sing, Watch, &amp; Learn Spanish (DVD + Guide): 16 Fun Music Videos to Introduce Your Child to Spanish",[],,McGraw Hill,[],"15,792 in Software (",[],Software,,</div>,,0071480935,"[https://images-na.ssl-images-amazon.com/images/I/51BtLn2-L-L._SX38_SY50_CR,0,0,38,50_.jpg]",[https://images-na.ssl-images-amazon.com/images/I/51BtLn2-L-L.jpg],NaN
2,[],,"<b>Connect is the only integrated learning system that empowers students by continuously adapting to deliver precisely what they need, when they need it, how they need it, so that your class time is more engaging and effective.</b><br />. Kelly Cowan just celebrated her 20th anniversary at Miami...",,Connect with LearnSmart Access Card for Microbiology Fundamentals: A Clinical Approach,[],,McGraw-Hill Science/Engineering/Math,[],"16,900 in Software (",[],Software,,</div>,,007329506X,"[https://images-na.ssl-images-amazon.com/images/I/51z1MKLSD3L._SX38_SY50_CR,0,0,38,50_.jpg]",[https://images-na.ssl-images-amazon.com/images/I/51z1MKLSD3L.jpg],NaN
4,[],,<i>Anatomy &amp; Physiology Revealed Cat</i> is the ultimate online interactive cat dissection experience. This state-of-the-art program uses cat photos combined with a layering technique that allows the student to peel away layers of the cat to reveal structures beneath the surface. <i>Anatomy ...,,Anatomy &amp; Physiology Revealed Student Access Card (Cat Version),"[0323394612, 0323227937, 1118527488]",,McGraw-Hill Education,[],"14,861 in Software (",[],Software,,</div>,$4.83,0073525758,"[https://images-na.ssl-images-amazon.com/images/I/51Do0BbcTkL._SX38_SY50_CR,0,0,38,50_.jpg]",[https://images-na.ssl-images-amazon.com/images/I/51Do0BbcTkL.jpg],NaN


In [124]:
# A lot of the descriptions (and other features) contain HTML.
# The function parses and "translates" into plain text descriptions more suitable for analysis

def strip_html(s):
    if not s or s.isspace(): 
        return ''
    try:
        #print(s)
        strr = str(html.fromstring(s).text_content())
        # strr = strr.replace('\n', '').replace('\t', '').replace("&nbsp", '').replace('\r', '')
        return strr
        # return str(html.fromstring(s).text_content(s))
    except etree.ParserError: # I am not able to find out why the error occur so i continued by catching the exception. Seem to happen on some empty description strings 
        return ''

# df2.iloc[0].description = strip_html(df2.iloc[0].description)
df2.description = df2.description.apply(lambda x: strip_html(x))
df2.iloc[0:2]
# df2.loc[0:2]

# dfdescription = df2.description
# len(dfdescription.iloc[0])

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
1,[],,"Latin rhythms that will get your kids singing in Spanish. Sing, Watch, and Learn Spanish helps your kids ages four through eight take a giant step in learning Spanish by combining two time-honored methods kids have always used to develop their language skills: Imitating other kids and singing al...",,"Sing, Watch, &amp; Learn Spanish (DVD + Guide): 16 Fun Music Videos to Introduce Your Child to Spanish",[],,McGraw Hill,[],"15,792 in Software (",[],Software,,</div>,,0071480935,"[https://images-na.ssl-images-amazon.com/images/I/51BtLn2-L-L._SX38_SY50_CR,0,0,38,50_.jpg]",[https://images-na.ssl-images-amazon.com/images/I/51BtLn2-L-L.jpg],NaN
2,[],,"Connect is the only integrated learning system that empowers students by continuously adapting to deliver precisely what they need, when they need it, how they need it, so that your class time is more engaging and effective.. Kelly Cowan just celebrated her 20th anniversary at Miami University M...",,Connect with LearnSmart Access Card for Microbiology Fundamentals: A Clinical Approach,[],,McGraw-Hill Science/Engineering/Math,[],"16,900 in Software (",[],Software,,</div>,,007329506X,"[https://images-na.ssl-images-amazon.com/images/I/51z1MKLSD3L._SX38_SY50_CR,0,0,38,50_.jpg]",[https://images-na.ssl-images-amazon.com/images/I/51z1MKLSD3L.jpg],NaN


In [125]:
len(np.where(df2.asin.duplicated())[0])

4403

### Does any product contain different descriptions?  

In [126]:
# Counting occurence of unique "asin"
asin_count = df2['asin'].value_counts()
# print(asin_count)
asin_more_than_once = asin_count[asin_count > 1].index
# print(asin_more_than_once)
# Step 2: Filter df2 to keep rows where 'asin' is in asin_more_than_once
filtered_df = df2[df2['asin'].isin(asin_more_than_once)]
filtered_df = filtered_df[["asin","description"]].sort_values(by="asin")


In [127]:
# Visual confirmation of duplicates 
filtered_df

,asin,description
877,B00000IV94,SET the Computer Game shares the game features of the world famous SET Game. The computer version offers up to ten players the opportunity to play together on one computer or solitaire or against the computer.. The object of the game is to identify sets of three cards. Each card is unique in its...
6028,B00000IV94,SET the Computer Game shares the game features of the world famous SET Game. The computer version offers up to ten players the opportunity to play together on one computer or solitaire or against the computer.. The object of the game is to identify sets of three cards. Each card is unique in its...
878,B00000IWIZ,"Star Wars Episode I: The Gungan Frontier is the first ever Star Wars ecological simulation game designed for players 10 and older. Taking place during the Episode I time period, players must transform the barren moon of Naboo into a thriving, balanced ecosystem and help the Gungans build a new u..."
6029,B00000IWIZ,"Star Wars Episode I: The Gungan Frontier is the first ever Star Wars ecological simulation game designed for players 10 and older. Taking place during the Episode I time period, players must transform the barren moon of Naboo into a thriving, balanced ecosystem and help the Gungans build a new u..."
879,B00000J0GM,"Magellan GPS 315 Datasend CD-ROM Data Management Software, transfers up to 20,000 points of interest data between GPS and Point.. Use the DataSend CD-ROM with your Magellan GPS 315 or GPS 320 global-positioning-system receiver to find North American points of interest. To use this product, first..."
...,...,...
6025,B0005MYHMU,"Discover the history of the Klondike Gold Rush! July 1897. The United States was in the throes of a depression, and people were beginning to lose hope. Then, from out of the North, came astonishing news: GOLD! The rush was on, as tens of thousands of people caught gold fever and set out for the ..."
6027,B0005MYI4W,From the Makers of The Oregon Trail Series Storybook Weaver DELUXE is packed with thousands of story-starting images to stimulate your creativity. Write in English or Spanish for hours of creative possibilities. And the text-to-speech feature lets you hear your story read aloud. Educational Be...
11178,B0005MYI4W,From the Makers of The Oregon Trail Series Storybook Weaver DELUXE is packed with thousands of story-starting images to stimulate your creativity. Write in English or Spanish for hours of creative possibilities. And the text-to-speech feature lets you hear your story read aloud. Educational Be...
6026,B0005MYJ0A,This CD software teaches the geography of Africa in an engaging and enjoyable computer learning environment.


In [128]:
# If "asin" and "description" match -> drop
filtered_df.drop_duplicates(inplace=True)

# How many unique "asin" ?
len(filtered_df.asin.unique())
filtered_df

,asin,description
877,B00000IV94,SET the Computer Game shares the game features of the world famous SET Game. The computer version offers up to ten players the opportunity to play together on one computer or solitaire or against the computer.. The object of the game is to identify sets of three cards. Each card is unique in its...
878,B00000IWIZ,"Star Wars Episode I: The Gungan Frontier is the first ever Star Wars ecological simulation game designed for players 10 and older. Taking place during the Episode I time period, players must transform the barren moon of Naboo into a thriving, balanced ecosystem and help the Gungans build a new u..."
879,B00000J0GM,"Magellan GPS 315 Datasend CD-ROM Data Management Software, transfers up to 20,000 points of interest data between GPS and Point.. Use the DataSend CD-ROM with your Magellan GPS 315 or GPS 320 global-positioning-system receiver to find North American points of interest. To use this product, first..."
881,B00000JIXG,"Academic version, microsoft office 2000.. Microsoft Office has established a position as the most efficient suite of applications for document creation, communication, and business information analysis. For many functions, the business platform has evolved from paper to the Web.. Microsoft Offic..."
880,B00000JIXM,"Professional Upgrade includes: Microsoft Word - word processor, Microsoft Excel - spreadsheets, Microsoft Publisher - desktop publishing, Microsoft Access - database management, Microsoft PowerPoint - presentation graphics, and many more features!. Microsoft Office has established a position as ..."
...,...,...
6024,B0004N37GW,Protect your child from inappropriate emailProduct InformationThe Software: You are never too young to plug in tothe wired world of e-communication and KidMail Safety is the perfect way to letchildren as young as 5 years old join the online conversation. With 5animated desktop themes featuring ...
11174,B0004O05WK,RollerCoaster Tycoon 2: Time Twister Expansion Pack
11176,B0005MYHMU,"Discover the history of the Klondike Gold Rush! July 1897. The United States was in the throes of a depression, and people were beginning to lose hope. Then, from out of the North, came astonishing news: GOLD! The rush was on, as tens of thousands of people caught gold fever and set out for the ..."
6027,B0005MYI4W,From the Makers of The Oregon Trail Series Storybook Weaver DELUXE is packed with thousands of story-starting images to stimulate your creativity. Write in English or Spanish for hours of creative possibilities. And the text-to-speech feature lets you hear your story read aloud. Educational Be...


Removing the duplicates

In [129]:
df_asin_description = df2[["asin","description"]].copy()
df_asin_description.drop_duplicates(inplace=True)
# print(len(df_asin_description))
df_asin_description

,asin,description
1,0071480935,"Latin rhythms that will get your kids singing in Spanish. Sing, Watch, and Learn Spanish helps your kids ages four through eight take a giant step in learning Spanish by combining two time-honored methods kids have always used to develop their language skills: Imitating other kids and singing al..."
2,007329506X,"Connect is the only integrated learning system that empowers students by continuously adapting to deliver precisely what they need, when they need it, how they need it, so that your class time is more engaging and effective.. Kelly Cowan just celebrated her 20th anniversary at Miami University M..."
4,0073525758,Anatomy & Physiology Revealed Cat is the ultimate online interactive cat dissection experience. This state-of-the-art program uses cat photos combined with a layering technique that allows the student to peel away layers of the cat to reveal structures beneath the surface. Anatomy & Physiology R...
5,0077340701,"John Coburn grew up in the Hawaiian Islands, the seventh of sixteen children. He received his Associate of Arts degree in 1977 from Windward Community College, where he graduated with honors. In 1979 he received a Bachelors Degree in Education from the University of Hawaii. After being lured int..."
6,0077369823,"Connect is the only integrated learning system that empowers students by continuously adapting to deliver precisely what they need, when they need it, how they need it, so that your class time is more engaging and effective.. Kelly Cowan just celebrated her 20th anniversary at Miami University M..."
...,...,...
26785,B01HEFZJC2,"Features: 1. Beautiful and fabulously detailed real silver necklace 2. Truly elegant and breathtaking design looks gorgeous and amazing 3. White Gold plated design, durable and long lasting 4. Easy to match and suitable for any style of clothes 5. Great in detail and good for your personal jewel..."
26786,B01HEFZKEE,"Features: 1. Beautiful and fabulously detailed real silver necklace 2. Truly elegant and breathtaking design looks gorgeous and amazing 3. White Gold plated design, durable and long lasting 4. Easy to match and suitable for any style of clothes 5. Great in detail and good for your personal jewel..."
26787,B01HF3G4BS,"Mac Internet Security X9 contains two of our best-selling security products that protect your Mac against malware and network attacks: Intego VirusBarrier X9 and Intego NetBarrier X9. Together they ensure your Mac is protected against adware, malware, strangers, and unknown applications trying t..."
26788,B01HF41TKI,"VersaCheck X9 for QuickBooks 2016 DNA Secure 100% compliant with US and Canadian bank requirements. Compatible: VersaCheck, VersaInk, VersaToner"


Creating shingles

In [130]:
def shingle(aString, q, delimiter=' '):
    """
    Input:
        - aString (str): string to split into shingles
        - q (int)
        - delimiter (str): string of the delimiter to consider to split the input string (default: space)
    Return: list of unique shingles
    """
    all_shingles = []
    if delimiter != '':
        words_list = aString.split(delimiter)
    else:
        words_list = aString
    for i in range (len(words_list)-q+1):
        all_shingles.append(delimiter.join(words_list[i:i+q]))
    return list(set(all_shingles))

In [131]:
# df_asin_description
# Apply shingles to the df_asin_description
df_asin_description["shingles"] = df_asin_description["description"].apply(lambda x: shingle(x, 2))
# df_asin_description

Computing similarity with Jaqquard approach

In [132]:
def similarity(intersection_set, union_set):
    return len(intersection_set)/len(union_set)


In [133]:
input = "Latin Rhythms for Kids is an engaging educational resource designed to facilitate Spanish language learning for children aged four to eight. The program utilizes a combination of two proven methods for language development: imitation and singing. The DVD comprises 16 music videos featuring children participating in enjoyable activities, such as zoo visits and exploring grandma's closet. Each video showcases original songs with authentic Latin rhythms, encouraging kids to sing along. The lyrics, consisting of 300 Spanish words, are displayed on screen as subtitles, providing a visual aid for language absorption. The DVD aims to make language learning entertaining and effective, leveraging the natural affinity children have for singing and imitation. The content is designed to capture children's attention, fostering a fun and immersive environment for language acquisition. Additionally, the program incorporates Connect, an integrated learning system that adapts to students' needs, making the learning process engaging and efficient.Latin Rhythms for Kids is a valuable tool for parents and educators seeking an interactive and enjoyable approach to introducing Spanish language skills to young children. Through a combination of music, visuals, and adaptability, the program aims to create a positive language learning experience."
input_user = shingle(input, 2)  
# print(input_user)
# print(df_asin_description.shingles.iloc[0])
intersection_jaqquard = set(input_user).intersection(set(df_asin_description.shingles.iloc[0]))
union_jaqquard = set(input_user).union(set(df_asin_description.shingles.iloc[0]))
# print(intersection_jaqquard)
# print(union_jaqquard)
sim = similarity(intersection_jaqquard, union_jaqquard)
print(sim)




0.024390243902439025


In [134]:
# df_asin_description
df_asin_description["similarity"] = df_asin_description["shingles"].apply(lambda x: similarity(set(input_user).intersection(set(x)), set(input_user).union(set(x))))
df_asin_description


,asin,description,shingles,similarity
1,0071480935,"Latin rhythms that will get your kids singing in Spanish. Sing, Watch, and Learn Spanish helps your kids ages four through eight take a giant step in learning Spanish by combining two time-honored methods kids have always used to develop their language skills: Imitating other kids and singing al...","[a giant, a song, in learning, language skills:, in a, giant step, out in, Imitating other, music videos, grandmas closet.., song and, displayed as, fun activities,, kids singing, get your, original song, videos featuring, kids ages, authentic Latin, ages four, kids engaged, song of, each of, to...",0.024390
2,007329506X,"Connect is the only integrated learning system that empowers students by continuously adapting to deliver precisely what they need, when they need it, how they need it, so that your class time is more engaging and effective.. Kelly Cowan just celebrated her 20th anniversary at Miami University M...","[deliver precisely, only integrated, pursuing an, it her, in dental, empowers students, to nonmajors,, Middletown, an, especially to, mission to, her Ph.D., to pre-nursing, 20th anniversary, more engaging, an undergraduate, effective.. Kelly, in love, just celebrated, that your, Groningen in, is...",0.013468
4,0073525758,Anatomy & Physiology Revealed Cat is the ultimate online interactive cat dissection experience. This state-of-the-art program uses cat photos combined with a layering technique that allows the student to peel away layers of the cat to reveal structures beneath the surface. Anatomy & Physiology R...,"[or can, course; Anatomy, of any, or two, and comprehensive, used as, & physiology, available stand-alone,, & Physiology, layers of, histologic and, Cat is, structures beneath, Jackie Butler, peel away, Physiology Revealed, radiologic imaging,, County College., Science at, Grayson County, physio...",0.000000
5,0077340701,"John Coburn grew up in the Hawaiian Islands, the seventh of sixteen children. He received his Associate of Arts degree in 1977 from Windward Community College, where he graduated with honors. In 1979 he received a Bachelors Degree in Education from the University of Hawaii. After being lured int...","[to make, engaging one, his family,, fifteen years,, Louis Community, St. Louis, the seventh, seek a, being lured, a Bachelors, John Coburn, to his, Florissant Valley, for five, the learning, Whos Who, been teaching, as an, seventh of, of sixteen, numerous presentations, position in, received hi...",0.005141
6,0077369823,"Connect is the only integrated learning system that empowers students by continuously adapting to deliver precisely what they need, when they need it, how they need it, so that your class time is more engaging and effective.. Kelly Cowan just celebrated her 20th anniversary at Miami University M...","[deliver precisely, only integrated, pursuing an, it her, in dental, empowers students, to nonmajors,, Middletown, an, especially to, mission to, her Ph.D., to pre-nursing, 20th anniversary, more engaging, an undergraduate, effective.. Kelly, in love, just celebrated, that your, Groningen in, is...",0.013468
...,...,...,...,...
26785,B01HEFZJC2,"Features: 1. Beautiful and fabulously detailed real silver necklace 2. Truly elegant and breathtaking design looks gorgeous and amazing 3. White Gold plated design, durable and long lasting 4. Easy to match and suitable for any style of clothes 5. Great in detail and good for your personal jewel...","[avoid contacting, 2.Please avoid, lot, shower, Great in, 4. Easy, long lasting, Ideal gift, 1.Please avoid, Beautiful and, anniversary, engagement,, etc 7., acid and, party, meeting,, looks gorgeous, to avoid, wipe them, match and, lasting 4., engagement, party,, good for, Warm tips:, jewelry c...",0.000000
26786,B01HEFZKEE,"Features: 1. Beautiful and fabulously detailed real silver necklace 2. Truly elegant and breathtaking design looks gorgeous and amazing 3. White Gold plated design, d

Dataframe sorted by similarity

In [135]:

df_asin_description.sort_values(by="similarity", ascending=False, inplace=True)
df_asin_description

,asin,description,shingles,similarity
1,0071480935,"Latin rhythms that will get your kids singing in Spanish. Sing, Watch, and Learn Spanish helps your kids ages four through eight take a giant step in learning Spanish by combining two time-honored methods kids have always used to develop their language skills: Imitating other kids and singing al...","[a giant, a song, in learning, language skills:, in a, giant step, out in, Imitating other, music videos, grandmas closet.., song and, displayed as, fun activities,, kids singing, get your, original song, videos featuring, kids ages, authentic Latin, ages four, kids engaged, song of, each of, to...",0.024390
1414,B00002S9GS,SCHOLASTIC'S The Magic School Bus Explores the Ocean [Old Version]. The Magic School Bus Explores the Ocean is an interactive science adventure that helps children learn about the wonders of the ocean as they join the Magic School Bus class to explore the seven ocean zones. Available for both Ma...,"[class to, (Ages 6-10), Ocean is, Macintosh and, in a, fun and, to entertain, and age-appropriate, about science, the wonders, the ocean, encourage a, learn about, technology. The, the seven, wonders of, and encourage, especially about, both Macintosh, the Magic, designed to, Ocean teaches, inte...",0.024291
1326,B00002S8C4,"KidSpeak is a language-learning program that makes learning French easyfor kids of all ages. It features an engaging combination of animation,activities, games, puzzles, and songs. The program is hosted by a cartoon""friend"" and his or her playful pet who use only the language being taught.It is ...","[all ages., time, and, of animation,activities,, Help inEnglish, use only, is hosted, puzzles, and, children correct, an engaging, tell time,, French easyfor, alphabet and, a language-learning, wordrecognition, simple, numbers,simple addition, subtraction, how, games, puzzles,, correct pronuncia...",0.024194
41,0077627008,"Connect is the only integrated learning system that empowers students by continuously adapting to deliver precisely what they need, when they need it, how they need it, so that your class time is more engaging and effective.. connect.mheducation.com. Arthur Getis is a retired professor from San ...","[retired professor, deliver precisely, precisely what, only integrated, so that, time is, a retired, need it,, what they, integrated learning, Getis is, need, when, professor from, effective.. connect.mheducation.com., empowers students, is more, Connect is, system that, engaging and, continuous...",0.022831
12120,B0009STM6Q,"JumpStart Phonics is an integrated learning system that teaches essential phonics skills--the building blocks of reading. With voice-recognition technology, children play fun learning games and receive immediate feedback as they hear, see, and speak phonetic sounds. Then, children enhance readin...","[games and, essential phonics, computer with, see, and, they hear,, hear, see,, that teaches, voice-recognition technology,, integrated learning, skills--the building, three skill-building, blocks of, phonics skills--the, as they, speak phonetic, learning games, system that, skills away, workboo...",0.022624
...,...,...,...,...
19145,B004XFFMSI,HDE EasyCap Model 002 - 4 Channel USB 2.0 DVR Video Audio CCTV Capture Adapter,"[4 Channel, Model 002, Audio CCTV, Channel USB, USB 2.0, CCTV Capture, 2.0 DVR, - 4, 002 -, Capture Adapter, HDE EasyCap, Video Audio, DVR Video, EasyCap Model]",0.000000
19146,B004XO6UYE,"Get a native DWG, high-performance CAD solutionwithout the high pricewith new CorelCAD. Open, work with and save files in DWG format for easy collaboration with colleagues, suppliers and partners.","[CAD solutionwithout, easy collaboration, in DWG, a native, work with, save files, files in, pricewith new, the high, Open, work, and partners., DWG format, collaboration with, suppliers and, with colleagues,, with and, CorelCAD. Open,, for easy, high-performance CAD, new CorelCAD., solutionwith...",0.000000
5950,B

In [144]:
# df_asin_description
f = open("demofile2.txt", "a")
for i in range(50):
    # print(df_asin_description.iloc[i].description)
    # print(type(df_asin_description.iloc[i].description))
    
    f.write(df_asin_description.iloc[i].description)
    # print("similarity: ", df_asin_description.iloc[i].similarity)
    # print("asin: ", df_asin_description.iloc[i].asin)
    # print("")
f.close()
print(df_asin_description.similarity.iloc[0:5000])

1        0.024390
1414     0.024291
1326     0.024194
41       0.022831
12120    0.022624
           ...   
4040     0.003846
25811    0.003846
855      0.003846
20260    0.003846
3074     0.003846
Name: similarity, Length: 5000, dtype: float64


In [137]:
# widgets.Text(value='', disabled=False)

In [138]:
# list_col = df_asin_description["description"].copy()
# out = list_col.to_json(orient='records')
# with open('file_name.txt', 'w') as f:
#     f.write(out)
# # print(list_col)
# # print(type(list_col))

In [139]:
# descr = defaultdict(list)
 
# for idx, row in df2.iterrows():
#     if row.description in descr[row.asin]:
#         print(idx, row.asin, row.description)
#     else:
#         descr[row.asin].append(row.description)


# import json 
    
# # Define student_details dictionary
    
# # Convert and write JSON object to file
# with open("sample.json", "w") as outfile: 
#     json.dump(descr, outfile)

In [140]:
# for key, value in descr.items():
    # print(type(value))
    # if len(elem.values())  >1:
    #     print(elem)d
    

In [141]:
# ex_string, q = dfdescription.iloc[0], 2
# ex_shingles = shingle(ex_string, q)
# # assert len(ex_shingles) == 7
# print('\nInitial string:', ex_string)
# print(f'>> Shingles with q = {q} :',ex_shingles)

In [142]:
# Merge description to reviews data using 'asin'

merged_df = df.merge(df2[['asin', 'description']], on='asin', how='left')